![](figures/Header.png)

##### Author:
M.Sc. Mike Lindner ([@Lindner](https://www.gpi.kit.edu/62_117.php))

# uniXtree inversion

## Import libaries

In [ ]:
# open libaries
import json
import matplotlib  
import os

# original functions
from py_src.Synthetics_Loader import Quick_Loader
from py_src.Sampling import uniXtree
from py_src.Postprocessing import plot_result_beachball, plot_result_waveforms, Cluster_Result
from py_src.util import *

## Load input JSON-file


For detailed infromation see Create JSON. 

In [ ]:
json_file = 'input_ev123_Example.json'

Load JSON and assign content to Container dictionary

In [ ]:
# location of main.ipynb
pwd = '/'.join(os.getcwd().split('/')[:-1])

with open(pwd+'/'+json_file) as json_data_file:
    Container = json.load(json_data_file)

Set shortcuts

In [ ]:
gen = Container['general']
path = Container['path']
netw = Container['network']
src = Container['source']
inv = Container['inversion']
pre = Container['preprocessing']

## Quick Loader

$\underline{\text{Input}}$:                   
**Container** dictionary       
Compare JSON_Creator for detailed information.  
    
$\underline{\text{Output}}$:                 
            
**SYNT**: a dictionary of fundamental synthetics. Its structure is as followed:             
<img src="figures/Flowchart_SYNT.png" width="500"/>   
The network object is representative of the station's keys. It is composed of an arbitrary number of stations structured as shown below for the observables (there is no network key)
In this example, we do only have one source id with F1_X0.
            
**OBS**: dictionary of observabels. Its Structure is as followed:    
<img src="figures/Flowchart_OBS.png" width="500"/>  
The [Trace object](https://docs.obspy.org/packages/obspy.core.html) is a standart obspy python object including the associated functions (filter, detrend, etc.). 

**inv_pre**: a dictionary of specific pre-inversion settings (e.g. S/N, timeshift, etc.)

Note: If you want to perform multiple runs with different settings (traces selection or filter) you need to reload the waveforms.

In [ ]:
from importlib import reload
import py_src.Synthetics_Loader as SL
reload(SL)

_, OBS, SYNT, Noise, inv_pre, _ = SL.Quick_Loader(Container)

### Change inversion settings

In [ ]:
Xtree = Container['inversion']['Xtree']
Xtree['starting_solution_F1'] = [180, 45, 0, 0,0] # [strike,dip,rake,clvd,iso]
Xtree['Xtree_range_F1'] = [180, 45, 180, 100, 0] # [dstrike,ddip,drake,dclvd,diso]
Xtree['full_space_Xlayer'] = 1
Xtree['top_fac'] = 4

### Inversion procedure

In [ ]:
inv['function'] = 'Bayesian_Cd'#'L2'#

### Construct data covariance matrix

In [ ]:
from importlib import reload
import py_src.Modeller as Mod
reload(Mod)


if inv['function'] != 'L2':
    Dcov = Mod.design_data_covariance_matrix(OBS,Container, # obs data and parameter container
                                             Noise_Dict=Noise, # dictionary of noise traces
                                             N_TSampl=150, # use only last N_TSampl (avoid filter effects at boundary)
                                             Noise_ampli=1.0, # multipy noise amplitude
                                             plot=False # display construction of cov. matrix
                                            )
    Container['inversion']['data_covariance'] = Dcov

### Run uniXtree sampling

In [ ]:
from importlib import reload
import py_src.Sampling as samp
reload(samp)


XTree = samp.uniXtree(Container=Container,Observed=OBS,Fundamentals=SYNT)
Result = XTree.organizer(show_progress=True)

### Create output folder

In [ ]:
out_folder = pwd+'/Figures/'
sub_folder = src['event_id']
path2output = create_output_folder(out_folder,sub_folder)

### Plot results

In [ ]:
if inv['function'] != 'L2':
    prob = [0.95,0.97]#
else:
    prob = [2,2]
plot_result_beachball(Result,inv_pre,Container,prob=prob)

### Cluster solutions

In [ ]:
from importlib import reload
import py_src.Postprocessing as post
reload(post)

cRes = post.Cluster_Result(Container=Container,OBS=OBS,SYNT=SYNT)
cRes.get_Kagan_cluster(Result,mKagan=[20.0,35.0])
cRes.display_cluster(fx='f1')
cl_dict1 = cRes.print_result(fx='f1',ev_suffix='10_10_10')

### Plot waveforms

In [ ]:
plot_result_waveforms(OBS,Result,Container,SYNT,inv_pre,
                      prob=10, # number (or <1 percentage) of solutions
                      display_sigma=True,
                      Noise_Dict=Noise,
                      N_TSampl=150,    
                      Noise_ampli=1.0
                      )